# The group task

- Loading packages

In [131]:
import numpy as np
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
import pyreadstat
import math

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

# To display the graphs here
%matplotlib inline


- Importing the data

In [25]:
health = pd.read_sas("/Users/nathankhadaroo/Desktop/task/Group/london ward data health.sas7bdat", format='sas7bdat', encoding='latin1')
environment = pd.read_csv("/Users/nathankhadaroo/Desktop/task/Group/London ward data environment.csv")
district_codes = pd.read_csv("/Users/nathankhadaroo/Desktop/task/Group/LondonDistrictcodes.csv")
demo = pd.read_table("/Users/nathankhadaroo/Desktop/task/Group/London ward data demographics.dat")
socio  = pd.read_csv("/Users/nathankhadaroo/Desktop/task/Group/lsd.csv")

In [63]:
district_codes.head()

,District,Districtcode
0,Barking and Dagenham,00AB
1,Barnet,00AC
2,Bexley,00AD
3,Brent,00AE
4,Bromley,00AF


In [27]:
demo.head()

,Wardname,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking
0,Hackney - Queensbridge,17.542063,8.2,44.7,38.3,11.8
1,Hammersmith & Fulham - S&s End,17.915361,8.4,30.6,39.0,11.8
2,Barking & Dagenham - River,26.851598,10.1,38.4,30.9,9.6
3,Tower Hamlets - Bethnal Green North,18.555872,7.0,49.4,38.7,15.0
4,Merton - Abbey,15.731861,8.5,26.6,38.3,12.5


In [28]:
environment.head()

,Wardcode,Population2011Census,Crimerate,Openspace
0,00ANGQ,11201,117.7,0.0
1,00ANGA,11518,114.0,0.3
2,00ADGN,10800,44.2,0.7
3,00BEGH,12321,65.3,0.7
4,00BCFZ,12609,64.3,1.3


In [29]:
socio.head()

,Wardcode,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,00ABFX,26.7,8.7,16.4,0.5
1,00ABFY,36.8,10.2,31.2,0.8
2,00ABFZ,29.4,10.2,28.0,0.9
3,00ABGB,20.0,8.8,29.9,1.1
4,00ABGC,37.4,10.2,28.9,0.8


In [30]:
health.head()

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy
0,Bromley - Darwin,5110.0,63.8,81.2,82.4
1,Kensington and Chelsea - Royal Hospital,7252.0,52.3,80.5,89.6
2,Hillingdon - Harefield,7399.0,55.8,78.3,82.3
3,Hammersmith and Fulham - Palace Riverside,7483.0,40.0,80.9,89.0
4,Kensington and Chelsea - Pembridge,7659.0,40.4,82.1,85.7


In [80]:
merge1 = pd.merge(health, demo)
merge1['Population2011Census'] = merge1['Population2011Census'].astype(int)
merge1.head()

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking
0,Bromley - Darwin,5110,63.8,81.2,82.4,18.317025,21.9,4.8,6.1,0.4
1,Hillingdon - Harefield,7399,55.8,78.3,82.3,19.245844,17.0,12.5,13.3,3.1
2,Westminster - Tachbrook,8158,33.3,82.2,83.1,10.750184,15.4,21.8,41.1,15.1
3,Merton - Village,8491,49.7,82.6,86.3,19.102579,19.3,17.1,34.0,8.3
4,Merton - Hillside,8977,60.0,81.0,91.7,13.824217,13.7,21.7,41.6,11.3


In [81]:
merge2 = pd.merge(environment, socio)
merge2.head()

,Wardcode,Population2011Census,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,00ANGQ,11201,117.7,0.0,23.7,3.6,9.4,0.6
1,00ANGA,11518,114.0,0.3,24.9,4.5,9.3,0.5
2,00ADGN,10800,44.2,0.7,1.1,2.9,22.1,1.3
3,00BEGH,12321,65.3,0.7,20.3,4.6,12.3,0.7
4,00BCFZ,12609,64.3,1.3,4.7,4.2,17.2,1.2


In [82]:
merged = pd.merge(merge1, merge2)

In [68]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 556 entries, 0 to 555
Data columns (total 17 columns):
Wardname                556 non-null object
Population2011Census    556 non-null int64
GeneralFertilityRate    556 non-null float64
Malelifeexpectancy      556 non-null float64
Femalelifeexpectancy    556 non-null float64
Children                556 non-null float64
Greaterthan65           556 non-null float64
nonwhite                556 non-null float64
NotBorninUK             556 non-null float64
NotEnglishspeaking      556 non-null float64
Wardcode                556 non-null object
Crimerate               556 non-null float64
Openspace               556 non-null float64
hhSocialRented          556 non-null float64
JobSeekers              556 non-null float64
Noqual                  556 non-null float64
Carsperhousehold        556 non-null float64
dtypes: float64(14), int64(1), object(2)
memory usage: 78.2+ KB


In [69]:
merged.head()

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,Wardcode,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold
0,Bromley - Darwin,5110,63.8,81.2,82.4,18.317025,21.9,4.8,6.1,0.4,00AFGQ,58.6,88.8,5.5,1.6,21.9,1.7
1,Hillingdon - Harefield,7399,55.8,78.3,82.3,19.245844,17.0,12.5,13.3,3.1,00ASGN,67.4,85.5,24.0,3.2,23.8,1.4
2,Westminster - Tachbrook,8158,33.3,82.2,83.1,10.750184,15.4,21.8,41.1,15.1,00BKGR,51.1,32.4,29.3,2.7,12.5,0.4
3,Merton - Village,8491,49.7,82.6,86.3,19.102579,19.3,17.1,34.0,8.3,00BAGQ,60.3,62.2,5.6,0.9,6.8,1.4
4,Merton - Hillside,8977,60.0,81.0,91.7,13.824217,13.7,21.7,41.6,11.3,00BAGE,52.0,5.7,9.9,1.2,6.4,0.8


In [71]:
merged.to_csv("/Users/nathankhadaroo/Desktop/task/Group/merged.csv")

Creating 

# Preparing the machine learning model

- We will prepare two models for both male and female life expectancy. First the menz.

#### Male model

In [75]:
X = merged[["Femalelifeexpectancy","Greaterthan65", "Noqual", "Crimerate", "JobSeekers", "Openspace"]]
X.head()

,Femalelifeexpectancy,Greaterthan65,Noqual,Crimerate,JobSeekers,Openspace
0,82.4,21.9,21.9,58.6,1.6,88.8
1,82.3,17.0,23.8,67.4,3.2,85.5
2,83.1,15.4,12.5,51.1,2.7,32.4
3,86.3,19.3,6.8,60.3,0.9,62.2
4,91.7,13.7,6.4,52.0,1.2,5.7


In [112]:
rf_raw = RandomForestRegressor(n_estimators=150, max_features=3)
rf_raw.fit(X, merged["Malelifeexpectancy"])
Male_pred = rf_raw.predict(X)

- Adding results to dataset 

In [156]:
df = pd.DataFrame(data=Male_pred, columns=["Male_pred"])
mergednew = pd.concat([merged, df], axis=1)
#mergednew

- Optimising parameters and calculating mse

In [145]:
param_grid = {"n_estimators": [5, 25, 50, 75, 100, 150],"max_features": [1, 2, 3, 4, 5, None]}
grid = GridSearchCV(RandomForestRegressor(), param_grid, scoring="neg_mean_squared_error", cv=5, n_jobs=-1)
grid.fit(X, merged["Malelifeexpectancy"])

/Users/nathankhadaroo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'n_estimators': [5, 25, 50, 75, 100, 150], 'max_features': [1, 2, 3, 4, 5, None]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [148]:
grid_res = pd.DataFrame(grid.cv_results_)
grid_res[grid_res["mean_test_score"] == grid_res["mean_test_score"].max()]["params"]

13    {'max_features': 3, 'n_estimators': 25}
Name: params, dtype: object

In [149]:
math.sqrt(-1*(grid_res[grid_res["mean_test_score"] == grid_res["mean_test_score"].max()]["mean_test_score"]))

4.812920063700243

#### Female model

In [151]:
X = merged[["Malelifeexpectancy","Greaterthan65", "Noqual", "Crimerate", "JobSeekers", "Openspace"]]
rf_raw = RandomForestRegressor(n_estimators=150, max_features=3)
rf_raw.fit(X, merged["Femalelifeexpectancy"])
Female_pred = rf_raw.predict(X)

- Adding to dataframe

In [159]:
df = pd.DataFrame(data=Female_pred, columns=["Female_pred"])
mergednew = pd.concat([mergednew, df], axis=1)
#mergednew

- Optimising parameters and calculating mse

In [162]:
grid.fit(X, merged["Femalelifeexpectancy"])
grid_res = pd.DataFrame(grid.cv_results_)
grid_res[grid_res["mean_test_score"] == grid_res["mean_test_score"].max()]["params"]

/Users/nathankhadaroo/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/nathankhadaroo/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nathankhadaroo/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:125: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/Users/nathankhad

11    {'max_features': 2, 'n_estimators': 150}
Name: params, dtype: object

In [163]:
math.sqrt(-1*(grid_res[grid_res["mean_test_score"] == grid_res["mean_test_score"].max()]["mean_test_score"]))

2.868716720260668

In [164]:
mergednew

,Wardname,Population2011Census,GeneralFertilityRate,Malelifeexpectancy,Femalelifeexpectancy,Children,Greaterthan65,nonwhite,NotBorninUK,NotEnglishspeaking,Wardcode,Crimerate,Openspace,hhSocialRented,JobSeekers,Noqual,Carsperhousehold,Male_pred,Female_pred,Female_pred
0,Bromley - Darwin,5110,63.8,81.2,82.4,18.317025,21.9,4.8,6.1,0.4,00AFGQ,58.6,88.8,5.5,1.6,21.9,1.7,80.672667,83.282667,83.282667
1,Hillingdon - Harefield,7399,55.8,78.3,82.3,19.245844,17.0,12.5,13.3,3.1,00ASGN,67.4,85.5,24.0,3.2,23.8,1.4,78.721333,82.992000,82.992000
2,Westminster - Tachbrook,8158,33.3,82.2,83.1,10.750184,15.4,21.8,41.1,15.1,00BKGR,51.1,32.4,29.3,2.7,12.5,0.4,81.465333,83.646667,83.646667
3,Merton - Village,8491,49.7,82.6,86.3,19.102579,19.3,17.1,34.0,8.3,00BAGQ,60.3,62.2,5.6,0.9,6.8,1.4,82.333333,86.846000,86.846000
4,Merton - Hillside,8977,60.0,81.0,91.7,13.824217,13.7,21.7,41.6,11.3,00BAGE,52.0,5.7,9.9,1.2,6.4,0.8,82.134000,89.690667,89.690667
5,Merton - Lower Morden,9090,67.7,80.1,86.6,19.185919,15.2,21.9,22.1,5.9,00BAGH,36.5,34.3,4.1,3.1,20.9,1.3,80.158000,85.760667,85.760667
6,Kingston upon Thames - Alexandra,9246,55.1,80.8,84.4,20.863076,14.8,29.8,27.2,8.2,00AXFX,49.2,43.5,4.9,2.1,15.8,1.4,80.642667,84.379333,84.379333
7,Merton - Cannon Hill,9249,66.9,79.7,83.6,17.731647,15.8,25.4,29.6,7.7,00BAFY,40.6,42.7,5.5,2.9,17.0,1.2,79.922667,83.741333,83.741333
8,Merton - Dundonald,9275,77.3,80.0,90.1,16.485175,9.8,18.1,35.6,7.3,00BAGB,79.0,15.2,3.7,1.4,7.8,0.8,81.034000,88.575333,88.575333
9,Westminster - Warwick,9369,40.7,80.7,95.1,10.716192,11.3,24.0,43.9,15.9,00BKGT,162.0,4.0,28.2,4.0,9.4,0.5,82.286667,92.168667,92.168667


# Visualising results 